In [1]:
import cv2 
import numpy as np
import torch
import torchvision
import matplotlib.pyplot as plt
from time import time
from torchvision import datasets, transforms
from torch import nn, optim

In [2]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Sequential(nn.Conv2d(in_channels=1,out_channels=16,kernel_size=5,stride=1,padding=2,)
                                   ,nn.ReLU()
                                   ,nn.MaxPool2d(kernel_size=2),)
        self.conv2 = nn.Sequential(nn.Conv2d(16, 32, 5, 1, 2),
                                   nn.ReLU(),
                                   nn.MaxPool2d(2),)
        # fully connected layer, output 10 classes
        self.out = nn.Linear(32 * 7 * 7, 10)
    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        # flatten the output of conv2 to (batch_size, 32 * 7 * 7)
        x = x.view(x.size(0), -1)       
        output = self.out(x)
        return output, x    # return x for visualization

model = CNN()

optimizer = optim.SGD(model.parameters(), lr=0.001) 
criterion = nn.NLLLoss() #Negative log loss function for n classes


model.load_state_dict(torch.load('/Users/admin/Digit_Detector.pt'))
model.eval()

CNN(
  (conv1): Sequential(
    (0): Conv2d(1, 16, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (conv2): Sequential(
    (0): Conv2d(16, 32, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (out): Linear(in_features=1568, out_features=10, bias=True)
)

In [3]:
load_from_sys = True

print("Code is running")
if load_from_sys:
    hsv_value = np.load('hsv_value.npy')
    print(hsv_value)

cap = cv2.VideoCapture(0)

if not cap.isOpened():
    print("Camera cannot be opened")
    exit()


cap.set(3,1280)
cap.set(4,720)

kernel = np.ones((5,5), np.uint8)
canvas = None

x1 = 0
y1 = 0

noise_thresh = 800


while True:
    ret, frame = cap.read()
    
    if not ret:
        print("Can't receive frame (stream end?). Exiting....")
        break
        
    if canvas is None:
        canvas = np.zeros_like(frame)
        
    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    
    if load_from_sys:
        lower_range = hsv_value[0]
        upper_range = hsv_value[1]
        
    mask = cv2.inRange(hsv,lower_range, upper_range)
    mask = cv2.erode(mask, kernel, iterations = 1)
    mask = cv2.dilate(mask, kernel, iterations = 2)
    
    contours, heirarchy = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
    
    if contours  and cv2.contourArea(max(contours, key = cv2.contourArea)) > noise_thresh:
        c = max(contours, key = cv2.contourArea)
        x2, y2 ,w, h = cv2.boundingRect(c)
        
        if x1 == 0 and y1 == 0:
            x1,y1 = x2,y2
        
        # Draw all contours
        # -1 signifies drawing all contours
        else:
            canvas = cv2.drawContours(frame, contours, -1, (0, 255, 0), 3)
            
        x1,y1 = x2,y2
        
    else:
        x1,y1 = 0, 0
        
    frame = cv2.add(frame, canvas)
    
    stacked = np.hstack((frame, canvas))
    cv2.imshow('Canny Edges After Contouring', mask)  
    cv2.imshow('Screen_Pen', cv2.resize(stacked, None, fx = 0.6, fy = 0.6))
  

    if cv2.waitKey(1) == ord(' '):
        blw = cv2.cvtColor(canvas, cv2.COLOR_BGR2GRAY)
        blw = cv2.resize(mask, (28, 28))
        blw = torch.Tensor(blw)
        blw = torch.Tensor.view(blw, (1, 1, 28, 28))
        y, x = model(blw)
        print(torch.max(y, 1)[1].data.squeeze())
        break


cap.release()
cv2.destroyAllWindows()

Code is running
[[ 41  55  77]
 [ 63 255 255]]
tensor(7)


##### 